# Imports libraries

In [1]:
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import cv2
import pandas_read_xml as pdx
import os
from sklearn.model_selection import train_test_split
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import zipfile
import dlib
from math import sqrt

# Create dataset

In [2]:
dfo = pd.DataFrame()
dfo["img_path"] = " "
dfo["x_max"] = " "
dfo["x_min"] = " "
dfo["y_min"] = " "
dfo["y_max"] = " "
dfo["resolution"] = " "
dfo["type"] = " "

In [3]:
numb_of_pic = len(os.listdir("./annotations")) 
tot = 0

for i in tqdm(range(numb_of_pic)):
    path_def = './annotations/maksssksksss'
    path = path_def + str(i) +'.xml'
    
    df = pdx.read_xml(path)
    for i in range(4):
        df = df.pipe(flatten)
    
    faces = df.shape[0]
    for j in range(faces):
        
        x_min = int(df['annotation|object|bndbox|xmin'][j])
        y_min = int(df['annotation|object|bndbox|ymin'][j])
        x_max = int(df['annotation|object|bndbox|xmax'][j])
        y_max = int(df['annotation|object|bndbox|ymax'][j])
        res = (x_max - x_min) * (y_max - y_min)
        path_to_img = df['annotation|filename'][j]
        
        tag = df['annotation|object|name'][j]
        
        dfo.loc[tot] = [path_to_img, x_max,x_min, y_min, y_max,res, tag]
        tot=tot+1


100%|██████████| 853/853 [00:28<00:00, 29.68it/s]


In [ ]:
dfo_grp = dfo.groupby('type').agg(['count','median'])

dfo_grp['resolution']
fig, axs = plt.subplots(1,figsize=(15,8))

sns.barplot(x=dfo_grp['resolution'].index,y=dfo_grp['resolution']['count'],ax=axs)

# Analisis

In [4]:
dfo = dfo[dfo.resolution >500]


dfo_grp = dfo.groupby('type').agg(['count','median'])
print('Resolusion media:','\n',dfo_grp['resolution']['median'],'\n')

Resolusion media: 
 type
mask_weared_incorrect    1800.0
with_mask                1331.0
without_mask             1221.0
Name: median, dtype: float64 



/tmp/ipykernel_107066/4196816882.py:4: FutureWarning: ['img_path'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  dfo_grp = dfo.groupby('type').agg(['count','median'])


In [ ]:
fig, axs = plt.subplots(1,figsize=(16,8))
sns.scatterplot(x=dfo.x_max - dfo.x_min ,y=dfo.y_max - dfo.y_min,ax=axs,hue=dfo.type,palette='husl')

In [6]:
labels= dfo.drop(['img_path',"x_max","x_min","y_min","y_max",'resolution'], axis=1)
data = dfo.drop(['type'], axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.2)

# Split data

In [ ]:
Data_dir = '../Data/3_class_init/'

In [ ]:
def save_into(x,y,base_dic,fol): 
    tot=0
    try:
        write_dic = base_dic + fol + '/'
        os.mkdir(base_dic+fol)
        os.mkdir(write_dic+'with_mask')
        os.mkdir(write_dic+'without_mask')
        os.mkdir(write_dic+'bad_mask')
    except:
        pass
        
    num = x.shape[0] -1
    
    for i in tqdm(range(num)):
        img_path = x.iloc[i][0]
        x_max = x.iloc[i][1]
        x_min = x.iloc[i][2]
        y_min = x.iloc[i][3]
        y_max = x.iloc[i][4]
        tag =y.iloc[i][0]
        
        img = cv2.imread('./images/'+img_path)
        crop = img[y_min:y_max, x_min:x_max]
        if(tag == 'with_mask'):
            cv2.imwrite(write_dic+ 'with_mask/' +str(tot)+'.png',crop)
        elif(tag == 'without_mask'):
            cv2.imwrite(write_dic+ 'without_mask/' +str(tot)+'.png',crop)
        else:
            cv2.imwrite(write_dic+ 'bad_mask/' +str(tot)+'.png',crop)
        tot =tot +1
        
save_into(x_train,y_train,Data_dir,'train')
save_into(x_test,y_test,Data_dir,'test')
save_into(x_val,y_val,Data_dir,'validation')

# Faces in the Wild

In [11]:
base = './lfw'
tot = 0
arr = os.listdir(base)
detector = dlib.get_frontal_face_detector()

for i in arr:
    per_dir = (base+'/'+i)
    person = os.listdir(per_dir)
    
    for j in person:
        img = cv2.imread(per_dir+'/'+j)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dets = detector(gray)
        
        for face in dets:
            x= face.left()
            y= face.top()
            w= face.width()
            h= face.height()
            crop = img[y:y+h, x:x+w]
            try:
                cv2.imwrite('./faces_wild/'+str(tot)+j, crop)
                tot = tot +1
            except:
                pass


In [13]:
arr = os.listdir('./faces_wild')
tot = 0

try(os.mkdir('../Data/best_data'))

for i in range(len(s.listdir(Data_dir + '/train/with_mask'))):
    img = cv2.imread(arr[tot])
    cv2.imwrite(Data_dir + '/train/without_mask/'+arr[tot],img)
    tot = tot +1
    
for i in range(len(s.listdir(Data_dir + '/test/with_mask'))):
    img = cv2.imread(arr[tot])
    cv2.imwrite(Data_dir + '/test/without_mask/'+arr[tot],img)
    tot = tot +1
    
for i in range(len(s.listdir(Data_dir + '/validation/with_mask'))):
    img = cv2.imread(arr[tot])
    cv2.imwrite(Data_dir + '/validation/without_mask/'+arr[tot],img)
    tot = tot +1

13552
